In [ ]:
#Libraries we need
import tensorflow as tf
#TFP bijector class and PDFs aff all sorts
import tensorflow_probability as tfp
import numpy as np
from tensorflow.keras.layers import Layer, Dense, ReLU
#Model to save the model otherwise I had problems
from tensorflow.keras import Model

#Shortnames
tfd = tfp.distributions
tfb = tfp.bijectors
K = tf.keras
#Change datatype here for extra precision
DTYPE=tf.float32

In [ ]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:

##NN layer inside the RNVP (scale and shift)
class NN(Layer):
  
  def __init__(self,input_shape,output_shape,n_hidden = [512,512],activation = 'relu', name = 'NN'):
    super(NN,self).__init__(name = name)
    layers = []
    for layer, parameters in enumerate(n_hidden):
      layers.append(Dense(parameters, activation = activation, name='dense_1_{}'.format(layer), kernel_initializer='glorot_normal',bias_initializer='zeros'))
    self.layer_list = layers
    #I played with initializers you might want to change them to something like uniform of normal
    self.s_layer = Dense(output_shape, activation ='tanh', name = 's',kernel_initializer='zeros',bias_initializer='zeros')
    self.t_layer = Dense(output_shape, activation = 'linear',name='t',kernel_initializer='zeros',bias_initializer='zeros')
    
  def call(self,x):
    input = x
    for layer in self.layer_list:
      input = layer(input)
    s = self.s_layer(input)
    t = self.t_layer(input)
    return s, t


#tfp realisazion of RNVP bijection layer Just put them in tfb Chain 
class RealNVP(tfb.Bijector):
  
  def __init__(self, input_shape, split = 2, n_hidden = [1024, 1024], validate_args=False, name = 'RealNVP'):
    super(RealNVP, self).__init__(
            forward_min_event_ndims=1,
            inverse_min_event_ndims=1,
            validate_args=validate_args,
            name=name
    )
    NN_input_shape = split[0]
    NN_output_shape = split[1]
    Network = NN(NN_input_shape,NN_output_shape, n_hidden)
    flowing_x = tf.keras.Input(NN_input_shape)
    s, t = Network(flowing_x)
    self.flowing_st = Model(flowing_x,[s,t],name='s_and_t_parametrized')
    self.split = split
  def _g_linear(self,x):
    s,t = self.flowing_st(x)
    return tfb.Chain([tfb.Shift(t),tfb.Scale(log_scale=s)])
  
  def _forward(self,x):
    x_1d, x_dD = tf.split(x,self.split,axis=-1)
    y_1d = x_1d
    y_dD = self._g_linear(x_1d).forward(x_dD)
    return tf.concat([y_1d,y_dD], axis = -1)
  
  def _inverse(self,y):
    y_1d, y_dD = tf.split(y,self.split,axis=-1)
    x_1d = y_1d
    x_dD = self._g_linear(y_1d).inverse(y_dD)
    return tf.concat([x_1d,x_dD], axis = -1)

  def _forward_log_det_jacobian(self, x):
    x_1d, x_dD = tf.split(x, self.split, axis=-1)
    return self._g_linear(x_1d).forward_log_det_jacobian(x_dD, event_ndims=1)

  def _inverse_log_det_jacobian(self, y):
    y_1d, y_dD = tf.split(y, self.split, axis=-1)
    return self._g_linear(y_1d).inverse_log_det_jacobian(y_dD, event_ndims=1)


#tfp realisazion of FFT bijection layer Just put them in tfb Chain 
class RealFFT(tfb.Bijector):
  
  def __init__(self, input_shape, validate_args=False, name = 'RealFFT',is_constant_jacobian=True):
    super(RealFFT, self).__init__(
            forward_min_event_ndims=1,
            inverse_min_event_ndims=1,
            validate_args=validate_args,
            name=name
    )
    self.input_shape = input_shape
  
  def _forward(self,x):
    x1,x1D,xD=tf.split(tf.signal.rfft(x),[1,self.input_shape//2-1,1],axis=-1)
    x1=tf.math.real(x1)
    x1D_real=tf.math.real(x1D)
    x1D_imag=tf.math.imag(x1D)
    xD=tf.math.real(xD)
    x = tf.concat([x1,x1D_real,x1D_imag,xD],axis=-1)
    return x
  
  def _inverse(self,y):
    y1, y1D_real, y1D_imag, yD = tf.split(y,[1,self.input_shape//2-1,self.input_shape//2-1,1],axis=-1)
    
    y1D_real = tf.concat([y1,y1D_real,yD],axis=-1)

    y1D_imag = tf.concat([0*y1,y1D_imag,0*yD],axis=-1)

    y = tf.signal.irfft(tf.complex(y1D_real,y1D_imag))

    return y

  def _inverse_log_det_jacobian(self, y):
    return -self._forward_log_det_jacobian(self._inverse(y))

  def _forward_log_det_jacobian(self, x):
    # The full log jacobian determinant would be tf.zero_like(x).
    # However, we circumvent materializing that, since the jacobian
    # calculation is input independent, and we specify it for one input.
    return tf.constant(42, x.dtype)




In [ ]:
#Permutation we use
#Here its just [1,2,3,4] to [1,3,2,4] this is done because latter we use second half as input for the s and t of the firts
input_size=144
perm = []
for i in np.arange(input_size//2,step=2):
  perm+=[i]+[input_size//2+i]

for i in np.arange(1,input_size//2,step=2):
  perm+=[i]+[input_size//2+i]
perm1 = perm
perm = []
for i in np.arange(input_size//2,input_size,step=1):
  perm+=[i]

for i in np.arange(input_size//2,step=1):
  perm+=[i]
perm2=perm
perm1[50]

mvn2 = tfd.MultivariateNormalDiag(loc=[0.]*input_size)

#number of bijection layers
num_realnvp = 16
bijector_chain = []
#sequence of RNVP and permutation layers
bijector_chain.append(tfp.bijectors.Invert(RealFFT(input_shape=input_size)))
bijector_chain.append(tfp.bijectors.Permute(perm1))
for i in range(num_realnvp):
    bijector_chain.append(tfp.bijectors.Permute(perm2))
    #network parameters
    bijector_chain.append(RealNVP(input_shape=input_size, split=[input_size//2,input_size//2],n_hidden=[256,256]))
bijector_chain.append(tfp.bijectors.Permute(perm1))
bijector_chain.append(RealFFT(input_shape=input_size))

flow2 = tfd.TransformedDistribution(
    distribution=mvn2,
    #we reverse the secuence because its actually applied first bijector in the list first otherwise its in a different direction
    bijector=tfb.Chain(list(reversed(bijector_chain)))
)
print('trainable_variables: ', len(flow2.bijector.trainable_variables))

In [ ]:
from tensorflow.keras.utils import Progbar

In [ ]:
#E=lambda*(x**2-f**2)**2 so f is the shift between minima 
#Start from one and increasy up to 5 
@tf.function
def Energy(y):
        lamd = 1.
        f = np.sqrt(1.2)
        a = 0.1
        m = 0.5
        kinetic_energy = tf.reduce_sum( m / ( 2 * a ) * (y-tf.roll(y,shift=-1,axis=1))**2,axis=-1)
        potential_energy = tf.reduce_sum(lamd * ( y**2 - f**2 )**2,axis=-1)
        return kinetic_energy + a*potential_energy


In [ ]:
# Training loop
n_epochs = 10
n_iter = 1000
n_samples = 4096 
#KLD xs is configuration Energy is the energy of this configuration log_p0 KL_D is some constants to which to grad is applied
#More or less they normalize the value and make procedure somehow more stable
#put them to zero for standart KLD                                
@tf.function
def KL_Divergence(xs,energy,log_p0,KL_D):
    log_p1 = flow2.log_prob(xs)
    KL_D1 = (log_p1+energy-KL_D)*tf.exp(log_p1-log_p0)/tf.abs(KL_D)
    KL_D = KL_D1
    return tf.math.reduce_mean(KL_D)
  
#these are optimizer and params
lr_decay = .1
learning_rate = .0001
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
checkpoint_directory = './'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=flow2)
for epoch in range(n_epochs):
    print('Epoch {:}/{:}'.format(epoch, n_epochs))
    checkpoint.save(checkpoint_directory+'ckpt_12_144_fft_{}'.format(epoch))  
    progbar = Progbar(n_iter)
    for iter in range(n_iter):
        accum_gradient = [tf.zeros_like(this_var) for this_var in flow2.trainable_variables]
        for k in range(1):
          xs_m = flow2.sample(n_samples)
          es_m = Energy(xs_m)
          log_p0 = flow2.log_prob(xs_m)
          KL_D=tf.reduce_mean(es_m+log_p0)
          with tf.GradientTape() as ae_tape:
              loss = KL_Divergence(xs_m,es_m,log_p0,KL_D)
          
          gradients = ae_tape.gradient(loss, flow2.trainable_variables)
          accum_gradient = [(acum_grad+grad) for acum_grad, grad in zip(accum_gradient, gradients)]
        
        accum_gradient = [this_grad/10. for this_grad in accum_gradient]
        optimizer.apply_gradients(zip(accum_gradient, flow2.trainable_variables))
        progbar.add(1, values=[('loss', KL_D)])

In [ ]:
#E=lambda*(x**2-f**2)**2 so f is the shift between minima 
#Start from one and increasy up to 5 
@tf.function
def Energy(y):
        lamd = 1.
        f = np.sqrt(1.3)
        a = 0.1
        m = 0.5
        kinetic_energy = tf.reduce_sum( m / ( 2 * a ) * (y-tf.roll(y,shift=-1,axis=1))**2,axis=-1)
        potential_energy = tf.reduce_sum(lamd * ( y**2 - f**2 )**2,axis=-1)
        return kinetic_energy + a*potential_energy
    
    
# Training loop
n_epochs = 10
n_iter = 1000
n_samples = 4096 
#KLD xs is configuration Energy is the energy of this configuration log_p0 KL_D is some constants to which to grad is applied
#More or less they normalize the value and make procedure somehow more stable
#put them to zero for standart KLD                                
@tf.function
def KL_Divergence(xs,energy,log_p0,KL_D):
    log_p1 = flow2.log_prob(xs)
    KL_D1 = (log_p1+energy-KL_D)*tf.exp(log_p1-log_p0)/tf.abs(KL_D)
    KL_D = KL_D1
    return tf.math.reduce_mean(KL_D)
  
#these are optimizer and params
lr_decay = .1
learning_rate = .0001
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
checkpoint_directory = './'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=flow2)
for epoch in range(n_epochs):
    print('Epoch {:}/{:}'.format(epoch, n_epochs))
    checkpoint.save(checkpoint_directory+'ckpt_13_144_fft_{}'.format(epoch))  
    progbar = Progbar(n_iter)
    for iter in range(n_iter):
        accum_gradient = [tf.zeros_like(this_var) for this_var in flow2.trainable_variables]
        for k in range(1):
          xs_m = flow2.sample(n_samples)
          es_m = Energy(xs_m)
          log_p0 = flow2.log_prob(xs_m)
          KL_D=tf.reduce_mean(es_m+log_p0)
          with tf.GradientTape() as ae_tape:
              loss = KL_Divergence(xs_m,es_m,log_p0,KL_D)
          
          gradients = ae_tape.gradient(loss, flow2.trainable_variables)
          accum_gradient = [(acum_grad+grad) for acum_grad, grad in zip(accum_gradient, gradients)]
        
        accum_gradient = [this_grad/10. for this_grad in accum_gradient]
        optimizer.apply_gradients(zip(accum_gradient, flow2.trainable_variables))
        progbar.add(1, values=[('loss', KL_D)])

In [ ]:
#E=lambda*(x**2-f**2)**2 so f is the shift between minima 
#Start from one and increasy up to 5 
@tf.function
def Energy(y):
        lamd = 1.
        f = np.sqrt(1.4)
        a = 0.1
        m = 0.5
        kinetic_energy = tf.reduce_sum( m / ( 2 * a ) * (y-tf.roll(y,shift=-1,axis=1))**2,axis=-1)
        potential_energy = tf.reduce_sum(lamd * ( y**2 - f**2 )**2,axis=-1)
        return kinetic_energy + a*potential_energy
    
    
# Training loop
n_epochs = 10
n_iter = 1000
n_samples = 4096 
#KLD xs is configuration Energy is the energy of this configuration log_p0 KL_D is some constants to which to grad is applied
#More or less they normalize the value and make procedure somehow more stable
#put them to zero for standart KLD                                
@tf.function
def KL_Divergence(xs,energy,log_p0,KL_D):
    log_p1 = flow2.log_prob(xs)
    KL_D1 = (log_p1+energy-KL_D)*tf.exp(log_p1-log_p0)/tf.abs(KL_D)
    KL_D = KL_D1
    return tf.math.reduce_mean(KL_D)
  
#these are optimizer and params
lr_decay = .1
learning_rate = .0001
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
checkpoint_directory = './'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=flow2)
for epoch in range(n_epochs):
    print('Epoch {:}/{:}'.format(epoch, n_epochs))
    checkpoint.save(checkpoint_directory+'ckpt_14_144_fft_{}'.format(epoch))  
    progbar = Progbar(n_iter)
    for iter in range(n_iter):
        accum_gradient = [tf.zeros_like(this_var) for this_var in flow2.trainable_variables]
        for k in range(1):
          xs_m = flow2.sample(n_samples)
          es_m = Energy(xs_m)
          log_p0 = flow2.log_prob(xs_m)
          KL_D=tf.reduce_mean(es_m+log_p0)
          with tf.GradientTape() as ae_tape:
              loss = KL_Divergence(xs_m,es_m,log_p0,KL_D)
          
          gradients = ae_tape.gradient(loss, flow2.trainable_variables)
          accum_gradient = [(acum_grad+grad) for acum_grad, grad in zip(accum_gradient, gradients)]
        
        accum_gradient = [this_grad/10. for this_grad in accum_gradient]
        optimizer.apply_gradients(zip(accum_gradient, flow2.trainable_variables))
        progbar.add(1, values=[('loss', KL_D)])

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
#E=lambda*(x**2-f**2)**2 so f is the shift between minima 
#Start from one and increasy up to 5 
@tf.function
def Energy(y):
        lamd = 1.
        f = np.sqrt(1.5)
        a = 0.1
        m = 0.5
        kinetic_energy = tf.reduce_sum( m / ( 2 * a ) * (y-tf.roll(y,shift=-1,axis=1))**2,axis=-1)
        potential_energy = tf.reduce_sum(lamd * ( y**2 - f**2 )**2,axis=-1)
        return kinetic_energy + a*potential_energy
    
    
# Training loop
n_epochs = 10
n_iter = 1000
n_samples = 4096 
#KLD xs is configuration Energy is the energy of this configuration log_p0 KL_D is some constants to which to grad is applied
#More or less they normalize the value and make procedure somehow more stable
#put them to zero for standart KLD                                
@tf.function
def KL_Divergence(xs,energy,log_p0,KL_D):
    log_p1 = flow2.log_prob(xs)
    KL_D1 = (log_p1+energy-KL_D)*tf.exp(log_p1-log_p0)/tf.abs(KL_D)
    KL_D = KL_D1
    return tf.math.reduce_mean(KL_D)
  
#these are optimizer and params
lr_decay = .1
learning_rate = .0001
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
checkpoint_directory = './'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=flow2)
for epoch in range(n_epochs):
    print('Epoch {:}/{:}'.format(epoch, n_epochs))
    checkpoint.save(checkpoint_directory+'ckpt_15_144_fft_{}'.format(epoch))  
    progbar = Progbar(n_iter)
    for iter in range(n_iter):
        accum_gradient = [tf.zeros_like(this_var) for this_var in flow2.trainable_variables]
        for k in range(1):
          xs_m = flow2.sample(n_samples)
          es_m = Energy(xs_m)
          log_p0 = flow2.log_prob(xs_m)
          KL_D=tf.reduce_mean(es_m+log_p0)
          with tf.GradientTape() as ae_tape:
              loss = KL_Divergence(xs_m,es_m,log_p0,KL_D)
          
          gradients = ae_tape.gradient(loss, flow2.trainable_variables)
          accum_gradient = [(acum_grad+grad) for acum_grad, grad in zip(accum_gradient, gradients)]
        
        accum_gradient = [this_grad/10. for this_grad in accum_gradient]
        optimizer.apply_gradients(zip(accum_gradient, flow2.trainable_variables))
        progbar.add(1, values=[('loss', KL_D)])

In [ ]:
#E=lambda*(x**2-f**2)**2 so f is the shift between minima 
#Start from one and increasy up to 5 
@tf.function
def Energy(y):
        lamd = 1.
        f = np.sqrt(1.6)
        a = 0.1
        m = 0.5
        kinetic_energy = tf.reduce_sum( m / ( 2 * a ) * (y-tf.roll(y,shift=-1,axis=1))**2,axis=-1)
        potential_energy = tf.reduce_sum(lamd * ( y**2 - f**2 )**2,axis=-1)
        return kinetic_energy + a*potential_energy
    
    
# Training loop
n_epochs = 50
n_iter = 1000
n_samples = 4096 
#KLD xs is configuration Energy is the energy of this configuration log_p0 KL_D is some constants to which to grad is applied
#More or less they normalize the value and make procedure somehow more stable
#put them to zero for standart KLD                                
@tf.function
def KL_Divergence(xs,energy,log_p0,KL_D):
    log_p1 = flow2.log_prob(xs)
    KL_D1 = (log_p1+energy-KL_D)*tf.exp(log_p1-log_p0)/tf.abs(KL_D)
    KL_D = KL_D1
    return tf.math.reduce_mean(KL_D)
  
#these are optimizer and params
lr_decay = .1
learning_rate = .0001
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
checkpoint_directory = './'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=flow2)
for epoch in range(n_epochs):
    print('Epoch {:}/{:}'.format(epoch, n_epochs))
    checkpoint.save(checkpoint_directory+'ckpt_16_144_fft_{}'.format(epoch))  
    progbar = Progbar(n_iter)
    for iter in range(n_iter):
        accum_gradient = [tf.zeros_like(this_var) for this_var in flow2.trainable_variables]
        for k in range(1):
          xs_m = flow2.sample(n_samples)
          es_m = Energy(xs_m)
          log_p0 = flow2.log_prob(xs_m)
          KL_D=tf.reduce_mean(es_m+log_p0)
          with tf.GradientTape() as ae_tape:
              loss = KL_Divergence(xs_m,es_m,log_p0,KL_D)
          
          gradients = ae_tape.gradient(loss, flow2.trainable_variables)
          accum_gradient = [(acum_grad+grad) for acum_grad, grad in zip(accum_gradient, gradients)]
        
        accum_gradient = [this_grad/10. for this_grad in accum_gradient]
        optimizer.apply_gradients(zip(accum_gradient, flow2.trainable_variables))
        progbar.add(1, values=[('loss', KL_D)])

In [ ]:
#E=lambda*(x**2-f**2)**2 so f is the shift between minima 
#Start from one and increasy up to 5 
@tf.function
def Energy(y):
        lamd = 1.
        f = np.sqrt(2)
        a = 0.1
        m = 0.5
        kinetic_energy = tf.reduce_sum( m / ( 2 * a ) * (y-tf.roll(y,shift=-1,axis=1))**2,axis=-1)
        potential_energy = tf.reduce_sum(lamd * ( y**2 - f**2 )**2,axis=-1)
        return kinetic_energy + a*potential_energy
    
    
# Training loop
n_epochs = 100
n_iter = 1000
n_samples = 4096 
#KLD xs is configuration Energy is the energy of this configuration log_p0 KL_D is some constants to which to grad is applied
#More or less they normalize the value and make procedure somehow more stable
#put them to zero for standart KLD                                
@tf.function
def KL_Divergence(xs,energy,log_p0,KL_D):
    log_p1 = flow2.log_prob(xs)
    KL_D1 = (log_p1+energy-KL_D)*tf.exp(log_p1-log_p0)/tf.abs(KL_D)
    KL_D = KL_D1
    return tf.math.reduce_mean(KL_D)
checkpoint_directory="C:\FFT\\"
#these are optimizer and params
lr_decay = .1
learning_rate = .0001
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=flow2)

#status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_directory))

for epoch in range(n_epochs):
    print('Epoch {:}/{:}'.format(epoch, n_epochs))
    checkpoint.save(checkpoint_directory+'ckpt_30_144_fft_{}'.format(epoch))  
    progbar = Progbar(n_iter)
    for iter in range(n_iter):
        accum_gradient = [tf.zeros_like(this_var) for this_var in flow2.trainable_variables]
        for k in range(1):
          xs_m = flow2.sample(n_samples)
          es_m = Energy(xs_m)
          log_p0 = flow2.log_prob(xs_m)
          KL_D=tf.reduce_mean(es_m+log_p0)
          with tf.GradientTape() as ae_tape:
              loss = KL_Divergence(xs_m,es_m,log_p0,KL_D)
          
          gradients = ae_tape.gradient(loss, flow2.trainable_variables)
          accum_gradient = [(acum_grad+grad) for acum_grad, grad in zip(accum_gradient, gradients)]
        
        accum_gradient = [this_grad/10. for this_grad in accum_gradient]
        optimizer.apply_gradients(zip(accum_gradient, flow2.trainable_variables))
        progbar.add(1, values=[('loss', KL_D)])